In [ ]:
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
# Define constants
data_path = "C:/Ashrit/Dataset_img"  # Replace with the actual path to your image data
image_size = 128

In [ ]:
# Function to load and preprocess image data
def load_data(path, max_samples=5000):
    images, ages = [], []
    all_images = os.listdir(path)[:max_samples]  # Limit for speed
    for img_name in all_images:
        img_path = os.path.join(path, img_name)
        
        try:
            # Assuming filename structure: nm0000100_rm1001569280_1955-1-6_2011.jpg
            parts = img_name.split("_")
            birth_date_str = parts[2]  # Expected birth date like "1955-1-6"
            image_year_str = parts[3].split(".")[0]  # Expected year like "2011"
            
            # Extract birth year and calculate age
            birth_year = int(birth_date_str.split("-")[0])
            image_year = int(image_year_str)
            age = image_year - birth_year
        except (IndexError, ValueError):
            print(f"Skipping {img_name} - couldn't extract age.")
            continue
        
        # Load and resize the image
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            images.append(img)
            ages.append(age)
        else:
            print(f"Skipping {img_name} - image not found or could not be read.")
    
    return np.array(images), np.array(ages)

# Load the dataset
images, ages = load_data(data_path)
images = images / 255.0  # Normalize images

# Check shapes of loaded data
print(f"Loaded {len(images)} images and {len(ages)} ages.")
print(f"Image shape: {images.shape}, Age shape: {ages.shape}")

In [ ]:
# Load VGG16 without top layers and freeze base layers
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))
base_model.trainable = False  # Freeze all layers

In [ ]:
# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='linear')(x)  # Linear activation for regression (age prediction)


In [ ]:
model = Model(inputs=base_model.input, outputs=x)

# Compile model with appropriate loss function and metrics
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])


In [ ]:
# Data augmentation generator
datagen = ImageDataGenerator(
    validation_split=0.2,  # Use 20% of data for validation
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ages, bins=20)
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter

# Convert ages to a list for easier counting
ages_list = list(ages)

# Combine images and ages to facilitate resampling
data = list(zip(images, ages_list))

# Identify the minority class
minority_class = [d for d in data if d[1] == min(set(ages_list), key=ages_list.count)]

# Ensure minority class has at least 2 samples by duplicating if necessary
if len(minority_class) < 2:
    data.extend(minority_class * (2 - len(minority_class)))

# Separate images and ages again
images, ages = zip(*data)
images = np.array(images)
ages = np.array(ages)

# Proceed with train-test split
images_train, images_val, ages_train, ages_val = train_test_split(
    images, ages, test_size=0.2, random_state=42, stratify=ages
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize ImageDataGenerator
datagen = ImageDataGenerator()

# Generate training and validation batches
train_generator = datagen.flow(images_train, ages_train, batch_size=32, shuffle=True)
val_generator = datagen.flow(images_val, ages_val, batch_size=32, shuffle=True)


In [ ]:
# Train the model
history = model.fit(train_generator, epochs=50, validation_data=val_generator, verbose=1)


In [ ]:
# Save the model
model.save("C:/Ashrit/ML Prjct 1/vgg16_age_detection_model.keras")
print("Model saved as vgg16_age_detection_model.keras")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training & validation MAE (Mean Absolute Error)
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Load the model for inference
from tensorflow.keras.models import load_model
import numpy as np

model = load_model("C:/Ashrit/ML Prjct 1/vgg16_age_detection_model.keras")

# Example prediction on a test image from the validation set
test_image = images_val[0]  # Use an example image from the validation set
test_image = np.expand_dims(test_image, axis=0)  # Expand dimensions for the model input

# Make prediction
predicted_age = model.predict(test_image)
print(f"Predicted Age: {predicted_age[0][0]}")
